In [3]:
!pip install -U langchain

     ---------------------------------------- 1.0/1.0 MB 7.1 MB/s eta 0:00:00
     ------------------------------------- 438.1/438.1 kB 13.8 MB/s eta 0:00:00
  Using cached langchain_text_splitters-0.3.8-py3-none-any.whl (32 kB)
     ------------------------------------- 363.0/363.0 kB 11.0 MB/s eta 0:00:00
     -------------------------------------- 444.2/444.2 kB 9.4 MB/s eta 0:00:00
     ---------------------------------------- 2.1/2.1 MB 8.4 MB/s eta 0:00:00
  Using cached tenacity-9.1.2-py3-none-any.whl (28 kB)
  Using cached jsonpatch-1.33-py2.py3-none-any.whl (12 kB)
  Using cached packaging-24.2-py3-none-any.whl (65 kB)
     -------------------------------------- 134.6/134.6 kB 8.3 MB/s eta 0:00:00
  Using cached requests_toolbelt-1.0.0-py2.py3-none-any.whl (54 kB)
  Using cached zstandard-0.23.0-cp311-cp311-win_amd64.whl (495 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl (13 kB)
     ---------------------------------------- 2.0/2.0 MB 12.5 MB/s eta 0:00:00
     ---


[notice] A new release of pip available: 22.3 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
!pip install langchain-openai

     ---------------------------------------- 65.2/65.2 kB 3.4 MB/s eta 0:00:00
     ------------------------------------- 725.5/725.5 kB 11.4 MB/s eta 0:00:00
  Using cached tiktoken-0.9.0-cp311-cp311-win_amd64.whl (893 kB)
  Using cached distro-1.9.0-py3-none-any.whl (20 kB)
     ---------------------------------------- 209.2/209.2 kB ? eta 0:00:00
  Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)
  Using cached regex-2024.11.6-cp311-cp311-win_amd64.whl (274 kB)



[notice] A new release of pip available: 22.3 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
!pip install -U langchain langchain-openai langchain-community chromadb pypdf

     ---------------------------------------- 2.5/2.5 MB 23.1 MB/s eta 0:00:00
     --------------------------------------- 19.3/19.3 MB 72.5 MB/s eta 0:00:00
     ------------------------------------- 304.2/304.2 kB 18.4 MB/s eta 0:00:00
     ---------------------------------------- 450.7/450.7 kB ? eta 0:00:00
  Using cached dataclasses_json-0.6.7-py3-none-any.whl (28 kB)
  Using cached pydantic_settings-2.9.1-py3-none-any.whl (44 kB)
  Using cached httpx_sse-0.4.0-py3-none-any.whl (7.8 kB)
     --------------------------------------- 13.0/13.0 MB 65.6 MB/s eta 0:00:00
  Using cached build-1.2.2.post1-py3-none-any.whl (22 kB)
  Using cached fastapi-0.115.9-py3-none-any.whl (94 kB)
     ---------------------------------------- 62.4/62.4 kB ? eta 0:00:00
     ---------------------------------------- 98.7/98.7 kB ? eta 0:00:00
     --------------------------------------- 12.7/12.7 MB 46.9 MB/s eta 0:00:00
     ---------------------------------------- 65.8/65.8 kB ? eta 0:00:00
     ----


[notice] A new release of pip available: 22.3 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [18]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain.chains.question_answering import load_qa_chain

In [19]:
from dotenv import load_dotenv
load_dotenv()
import os

In [20]:
os.environ["OPENAI_API_KEY"]= os.getenv("OPENAI_API_KEY")

In [21]:
#Load dos modelos (Embeddings e LLM)

embeddings_model = OpenAIEmbeddings()
llm = ChatOpenAI (model_name= "gpt-3.5-turbo", max_tokens = 200)

In [22]:
#Carregar o PDF

pdf_link = "CV-EDUARDO-FULLSTACK-ENGINEER-EN.pdf"

loader = PyPDFLoader(pdf_link, extract_images=False)
pages = loader.load_and_split()

In [23]:
#Separar em Chunks (Pedaços de documentos)

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 2000,
    chunk_overlap = 20,
    length_function = len,
    add_start_index = True
)

chunks = text_splitter.split_documents(pages)

In [28]:
!pip install openai --upgrade


[notice] A new release of pip available: 22.3 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [24]:
#Salvar no Vector DB - Chroma

db = Chroma.from_documents(chunks, embedding=embeddings_model, persist_directory="text_index")
db.persist()

In [25]:
#Carregar DB
vectordb = Chroma(persist_directory="text_index", embedding_function=embeddings_model)

#Load Retriever
retriver = vectordb.as_retriever(search_kwargs={"k": 3})

#Construção da cadeia de prompt para chamada do LLM
chain = load_qa_chain(llm, chain_type="stuff")

In [30]:
def ask(question):
    context = retriver.get_relevant_documents(question)

    #Coleta as fontes dos documentos
    sources = []
    for doc in context:
        source = doc.metadata.get("source", "Unknow source")
        sources.append(source)

    # Chamada ao LLM com contexto
    answer = (chain({"input_documents": context, "question": question}, return_only_outputs=True)) ['output_text']

    # Juntar fontes únicas e formatar para exibição
    unique_sources = list(set(sources))
    sources_text = "\n\nSources:\n" + "\n".join(unique_sources)
    
    return answer + sources_text

In [31]:
user_question = input("User: ")
answer = ask(user_question)
print("Answer: ", answer)

User:  Quais os principais frameworks utilizado por Carlos Eduardo?


Answer:  Os principais frameworks utilizados por Carlos Eduardo Barbosa são React.js, Next.js, Node.js, React Native, Angular, e Nest.js.

Sources:
CV-EDUARDO-FULLSTACK-ENGINEER-EN.pdf


In [16]:
!pip freeze > requirements.txt

In [32]:
def get_candidate_experience(company: str):
    experiences = {
        "LUSATI TECNOLOGIA": "Full Stack Engineer with React and Node, working on automation and performance improvements.",
        "OTHER COMPANY": "Some placeholder experience."
    }

    return experiences.get(company, "No experience found for this company.")

In [36]:
from openai import OpenAI
from langchain.schema import SystemMessage, HumanMessage
import json

llm_openai = ChatOpenAI(model_name="gpt-4", temperature=0, functions=[{
    "name": "get_candidate_experience",
    "description": "Fetches candidate's experience in a company",
    "parameters": {
        "type": "object",
        "properties": {
            "company": {
                "type": "string",
                "description": "Name of the company"
            }
        },
        "required": ["company"]
    }
}])

C:\Users\Usuário\.pyenv\pyenv-win\versions\3.11.0\Lib\site-packages\IPython\core\interactiveshell.py:3612: UserWarning: WARNING! functions is not default parameter.
                functions was transferred to model_kwargs.
                Please confirm that functions is what you intended.
  if await self.run_code(code, result, async_=asy):


In [39]:
response = llm_openai.predict_messages([
    SystemMessage(content="You are a job assistant AI."),
    HumanMessage(content="What was Carlos Eduardo’s role at LUSATI TECNOLOGIA?")
])

# Verifica se houve chamada de função
if hasattr(response, "additional_kwargs") and "function_call" in response.additional_kwargs:
    function_call = response.additional_kwargs["function_call"]
    function_name = function_call["name"]
    arguments = json.loads(function_call["arguments"])

    if function_name == "get_candidate_experience":
        result = get_candidate_experience(**arguments)
        print("Function result:", result)
else:
    print("LLM Response:", response.content)

Function result: Full Stack Engineer with React and Node, working on automation and performance improvements.
